In [ ]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from datetime import datetime
from pathlib import Path
from tqdm import tqdm 
import json
from pprint import pprint

sys.path.append(str(Path("..").resolve()))
from src import *

# Read Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [3]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/4_consistent/omero_museum"

In [4]:
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]: _id capacity duration enrolled room startDate ticketIds workshopTitle
[artworks]: _id authorIds comments_star_1 comments_star_2 comments_star_3 comments_star_4 comments_star_5 date description donationState donatorId isOriginal locationName materials period sellerId size tecniques trade type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[employees]: _id birth_date curriculum date_start email gender hometown name phone_number surname
[limited_events]: _id artist artworkTitles authorIds capacity description endDate roomName startDate theme ticketIds type
[messages]: _id content customerId deliveryStatus length sentDate type uri
[rooms]: _id floor
[suppliers]: _id email iban isMuseum isState name phoneNum surveys
[visitors]: _id birthDate chat comments donationIds email gender impairment isCustomer land name phoneNum saleIds surname surveys 

## Artworks

### RD1: _Read all the information of a given artwork, including its authors, materials, crafting techniques and room placement_

In [ ]:
artwork = "Mona Lisa"
doc = db.artworks.find_one({"_id": artwork})
pprint(doc,width=120)

### RD2: _Read the last 10 comments of a given artwork_

In [ ]:
pipeline = [
    {"$match": {"_id": "Mona Lisa"}},
    {"$project": {
        "all_comments": {
            "$concatArrays": [
                "$comments_star_1",
                "$comments_star_2",
                "$comments_star_3",
                "$comments_star_4",
                "$comments_star_5"
            ]
        }
    }},
    {"$unwind": "$all_comments"},
    {"$sort": {"all_comments.date": -1}},
    {"$limit": 10},
    {"$replaceRoot": {"newRoot": "$all_comments"}}
]

last_10_comments = list(db.artworks.aggregate(pipeline))
pprint(last_10_comments, width=120)

### RD3: _Calculate the average rating of a given artwork_

In [12]:
pipeline_avg = [
    {"$match": {"_id": "Mona Lisa"}},
    {"$project": {
        "all_comments": {
            "$concatArrays": [
                "$comments_star_1",
                "$comments_star_2",
                "$comments_star_3",
                "$comments_star_4",
                "$comments_star_5"
            ]
        }
    }},
    {"$unwind": "$all_comments"},
    {"$group": {
        "_id": None,
        "avg_rating": {"$avg": "$all_comments.rating"}
    }}
]

avg_rating_result = list(db.artworks.aggregate(pipeline_avg))
print("Average rating:", avg_rating_result[0]["avg_rating"] if avg_rating_result else None)

Average rating: 3.0


### RD4: _Calculate the average rating of the comments posted by a given customer_

In [14]:
customer_id = 0

pipeline = [
    {"$match": {"_id": customer_id}},
    {"$unwind": "$comments"},
    {"$group": {
        "_id": "$_id",
        "avg_rating": {"$avg": "$comments.rating"}
    }}
]

result = list(db.visitors.aggregate(pipeline))
print("Average rating by customer:", result[0]["avg_rating"] if result else None)

Average rating by customer: 3.3333333333333335


### RD5: _Read all the comments of a given artwork with a given rating_

In [ ]:
artwork_id = "Mona Lisa"
desired_rating = 4

pipeline = [
    {"$match": {"_id": artwork_id}},
    {"$project": {
        "all_comments": {
            "$concatArrays": [
                "$comments_star_1",
                "$comments_star_2",
                "$comments_star_3",
                "$comments_star_4",
                "$comments_star_5"
            ]
        }
    }},
    {"$unwind": "$all_comments"},
    {"$match": {"all_comments.rating": desired_rating}},
    {"$replaceRoot": {"newRoot": "$all_comments"}}
]

comments_with_rating = list(db.artworks.aggregate(pipeline))
pprint(comments_with_rating, width=120)

## Surveys

### RD6: _Read all the surveys, relative to this museum, filled out in a specific year_

In [ ]:
museum_id = 12
year = 2021

pipeline = [
    {"$match": {"_id": museum_id, "isMuseum": True}},
    {"$unwind": "$surveys"},
    {"$addFields": {
        "survey_year": {"$year": "$surveys.date_of_compilation"}
    }},
    {"$match": {"survey_year": year}},
    {"$replaceRoot": {"newRoot": "$surveys"}}
]

surveys_in_year = list(db.suppliers.aggregate(pipeline))
pprint(surveys_in_year, width=120)

### RD7: _Read all the surveys filled out by a specific customer_

In [ ]:
customer_id = 0

pipeline = [
    {"$match": {"_id": customer_id, "isCustomer": True}},
    {"$unwind": "$surveys"},
    {"$replaceRoot": {"newRoot": "$surveys"}}
]

customer_surveys = list(db.visitors.aggregate(pipeline))
pprint(customer_surveys, width=120)

### RD8: _Read all the surveys from a specific affiliated museum_

In [ ]:
affiliated_museum_id = 12

pipeline = [
    {"$match": {"_id": affiliated_museum_id, "isMuseum": True}},
    {"$unwind": "$surveys"},
    {"$replaceRoot": {"newRoot": "$surveys"}}
]

affiliated_museum_surveys = list(db.suppliers.aggregate(pipeline))
pprint(affiliated_museum_surveys, width=120)

## Tickets

### RD9: _Read all tickets of a visitor_

In [ ]:
visitor_id = 0

pipeline = [
    {"$match": {"_id": visitor_id}},
    {"$unwind": "$tickets"},
    {"$replaceRoot": {"newRoot": "$tickets"}}
]

visitor_tickets = list(db.visitors.aggregate(pipeline))
pprint(visitor_tickets, width=120)

### RD10: _Read all tickets for an event_

### RD11: _Count tickets issued in a year_

In [39]:
year = 2025

pipeline = [
    {"$unwind": "$tickets"},
    {"$match": {
        "tickets.date": {
            "$gte": datetime(year, 1, 1),
            "$lt": datetime(year + 1, 1, 1)
        }
    }},
    {"$count": "tickets_issued"}
]

result = list(db.visitors.aggregate(pipeline))
print(result)


[{'tickets_issued': 104}]


## Chats

### RD12: _Read 10 messages of a given chat previous than a specific date_

In [ ]:
customer_id = 0
date_limit = datetime(2025, 8, 25, 0, 0, 0)

pipeline = [
    {"$match": {"_id": customer_id}},
    {"$unwind": "$chat.new_messages"},
    {"$match": {"chat.new_messages.sentDate": {"$lt": date_limit}}},
    {"$sort": {"chat.new_messages.sentDate": -1}},
    {"$limit": 10},
    {"$replaceRoot": {"newRoot": "$chat.new_messages"}}
]

messages = list(db.visitors.aggregate(pipeline))
pprint(messages, width=120)

### RD13: _Read active chats for a guide_ (correction json first)

### RD14: _Count chats opened in a year_

In [24]:
year = 2025

pipeline = [
    {"$match": {"isCustomer": True, "chat.date_creation": {"$exists": True}}},
    {"$addFields": {"chat_year": {"$year": "$chat.date_creation"}}},
    {"$match": {"chat_year": year}},
    {"$count": "opened_chats"}
]

result = list(db.visitors.aggregate(pipeline))
print("Chats opened in", year, ":", result[0]["opened_chats"] if result else 0)

Chats opened in 2025 : 12


## Trades

### RD15: _Retrieve the artwork of a trade_

### RD16: _Count trades in a given year_

In [25]:
trade_year = 2025

pipeline_count_trades = [
    {"$match": {"trade.date_start": {"$exists": True}}},
    {"$addFields": {"trade_year": {"$year": "$trade.date_start"}}},
    {"$match": {"trade_year": trade_year}},
    {"$count": "trades_in_year"}
]

trades_in_year = list(db.artworks.aggregate(pipeline_count_trades))
print("Trades in", trade_year, ":", trades_in_year[0]["trades_in_year"] if trades_in_year else 0)

Trades in 2025 : 20


### RD17: _Retrieve the customer of a given artwork_

### RD18: _Retrieve all artwork inside a given room_

### RD19: _Retrieve all artwork of a given author_

In [ ]:
author_id = "68b572f8845fcdbabc1c8905"
artworks_by_author = list(db.artworks.aggregate([
    {"$match": {"authorIds": author_id}}
]))
pprint(artworks_by_author, width=120)

### RD20: _Retrieve all activity of a given workshop type_

In [ ]:
workshop_title = "Uno solo... ma tanti"
workshop_activities = list(db.activities.aggregate([
    {"$match": {"workshopTitle": workshop_title}}
]))
pprint(workshop_activities, width=120)

### RD21: _Retrieve all the employee of a given department_

### RD22: _Read all the surveys filled out in a specific day_

In [ ]:
specific_day = datetime(2021, 4, 22)

pipeline = [
    {"$unwind": "$surveys"},
    {"$addFields": {
        "survey_date": {
            "$dateToString": {"format": "%Y-%m-%d", "date": "$surveys.date_of_compilation"}
        }
    }},
    {"$match": {"survey_date": specific_day.strftime("%Y-%m-%d")}},
    {"$replaceRoot": {"newRoot": "$surveys"}}
]

surveys_on_day = list(db.suppliers.aggregate(pipeline))
pprint(surveys_on_day, width=120)

### RD23: _Retrive information about a given laboratory activity, including the total number of reservations_

In [ ]:
activity_id = 0

pipeline = [
    {"$match": {"_id": activity_id}},
    {"$addFields": {"total_reservations": {"$size": "$ticketIds"}}}
]

activity_info = list(db.activities.aggregate(pipeline))
pprint(activity_info, width=120)

### RD24: _View details about an employee, including their work shifts_

### RD25: _Retrive all the information about a limited event_ 

In [ ]:
event_id = "Digital Preservation of Cultural Heritage"
event_info = db.limited_events.find_one({"_id": event_id})
pprint(event_info, width=120)

### RD26: _Retrive all the details of a visitor_ 

In [ ]:
visitor_id = 0
visitor_info = db.visitors.find_one({"_id": visitor_id})
pprint(visitor_info, width=120)

### RD27: _Retrive the visitor count for each land from most frequent to least frequent_ 

In [ ]:
pipeline = [
    {"$group": {
        "_id": "$land",
        "visitorCount": {"$sum": 1}
    }},
    {"$sort": {"visitorCount": -1}}
]

result = list(db.visitors.aggregate(pipeline))
pprint(result, width=120)

### RD28: _Retrive the visitor count for each impairment from most frequent to least frequent_ 

In [ ]:
pipeline = [
    {"$group": {
        "_id": "$impairment",
        "visitorCount": {"$sum": 1}
    }},
    {"$sort": {"visitorCount": -1}}
]

result = list(db.visitors.aggregate(pipeline))
pprint(result, width=120)

### RD29: _Retrive the artwork count for each author from most frequent to least frequent_ 

### RD30: _Count the average age of participants at a given activity who completed the survey_ 

### RD31: _Count tickets issued for each year_ 

In [40]:
pipeline = [
    {"$unwind": "$tickets"},
    {"$group": {
        "_id": {"year": {"$year": "$tickets.date"}},
        "tickets_issued": {"$sum": 1}
    }},
    {"$sort": {"_id.year": 1}}
]

result = list(db.visitors.aggregate(pipeline))
print(result)

[{'_id': {'year': 2025}, 'tickets_issued': 104}]


### RD32: _Calculate the distribution of visitor ages grouped into bands based on ticket activity in a specific year_ 

In [44]:
from datetime import datetime

year = 2025
start_date = datetime(year, 1, 1)
end_date = datetime(year + 1, 1, 1)

pipeline = [
    {"$unwind": "$tickets"},
    {"$match": {
        "tickets.date": {"$gte": start_date, "$lt": end_date},
        "birthDate": {"$ne": None}
    }},
    {"$addFields": {
        "age": {
            "$dateDiff": {
                "startDate": "$birthDate",
                "endDate": "$tickets.date",
                "unit": "year"
            }
        }
    }},
    {"$addFields": {
        "age_group": {
            "$switch": {
                "branches": [
                    {"case": {"$lte": ["$age", 18]}, "then": "0-18"},
                    {"case": {"$and": [{"$gt": ["$age", 18]}, {"$lte": ["$age", 30]}]}, "then": "19-30"},
                    {"case": {"$and": [{"$gt": ["$age", 30]}, {"$lte": ["$age", 45]}]}, "then": "31-45"},
                    {"case": {"$and": [{"$gt": ["$age", 45]}, {"$lte": ["$age", 65]}]}, "then": "46-65"},
                    {"case": {"$gt": ["$age", 65]}, "then": "65+"}
                ],
                "default": "unknown"
            }
        }
    }},
    {"$group": {
        "_id": "$age_group",
        "count": {"$sum": 1}
    }},
    {"$sort": {"_id": 1}}
]

result = list(db.visitors.aggregate(pipeline))
for group in result:
    print(f"Age group {group['_id']}: {group['count']} visitors")


Age group 0-18: 104 visitors


### RD33: _Count material usage in all artworks_ 

In [45]:
pipeline = [
    {"$unwind": "$materials"},
    {"$group": {
        "_id": "$materials",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]

result = list(db.artworks.aggregate(pipeline))
for material in result:
    print(f"{material['_id']}: {material['count']} artworks")


Stone: 9 artworks
Canvas: 8 artworks
Clay: 6 artworks
Paper: 5 artworks
White Marble: 5 artworks
Glass: 4 artworks
Textile: 3 artworks
Bronze: 3 artworks
Wood: 2 artworks
Steel: 2 artworks


### RD33: _Count top 5 visit reasons_ 

In [48]:
pipeline = [
    {"$unwind": "$surveys"},
    {"$group": {
        "_id": "$surveys.reason_for_visit",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}},
    {"$limit": 5}
]

result = list(db.visitors.aggregate(pipeline))
for reason in result:
    print(f"{reason['_id']}: {reason['count']} responses")


trascorrere tempo libero con amici/parenti: 17 responses
passare un momento personale piacevole: 16 responses
incontrare persone con interessi simili ai miei: 14 responses
vedere oggetti importanti: 13 responses
approfondire le mie conoscenze: 9 responses


### RD34: _Count how many artworks are original vs. not_

In [50]:
pipeline = [
    {"$group": {
        "_id": "$isOriginal",
        "count": {"$sum": 1}
    }}
]

result = list(db.artworks.aggregate(pipeline))
for item in result:
    label = "Original" if item["_id"] else "Replica"
    print(f"{label}: {item['count']} artworks")


Replica: 15 artworks
Original: 5 artworks


### RD35: _Count how many visitors came back_

In [52]:
pipeline = [
    {"$project": {
        "num_tickets": {"$size": "$tickets"}
    }},
    {"$match": {
        "num_tickets": {"$gt": 1}
    }},
    {"$count": "repeat_visitors"}
]

result = list(db.visitors.aggregate(pipeline))
print("Repeat visitors:", result[0]["repeat_visitors"])


Repeat visitors: 32


### RD36: _Count which days had the most ticket activity_

In [53]:
pipeline = [
    {"$unwind": "$tickets"},
    {"$group": {
        "_id": {
            "$dateToString": {"format": "%Y-%m-%d", "date": "$tickets.date"}
        },
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}},
    {"$limit": 5}
]

result = list(db.visitors.aggregate(pipeline))
for day in result:
    print(f"{day['_id']}: {day['count']} tickets")


2025-09-04: 8 tickets
2025-09-01: 7 tickets
2025-09-03: 7 tickets
2025-08-22: 6 tickets
2025-08-16: 6 tickets


### RD37: _Average enrollment rate by workshop title_

In [59]:
pipeline = [
    {"$project": {
        "workshopTitle": 1,
        "enrollment_rate": {
            "$cond": [
                {"$eq": ["$capacity", 0]},
                0,
                {"$divide": ["$enrolled", "$capacity"]}
            ]
        }
    }},
    {"$group": {
        "_id": "$workshopTitle",
        "average_enrollment_rate": {"$avg": "$enrollment_rate"}
    }}
]

result = list(db.activities.aggregate(pipeline))
for activity in result:
    print(f"{activity['_id']}: {activity['average_enrollment_rate']:.2%} enrolled")


Ri-tratto con tatto: 40.91% enrolled
Libri tattili: 4.44% enrolled
Bestiario immaginario: 8.82% enrolled
Uno solo... ma tanti: 23.73% enrolled
Impronte: 21.35% enrolled
Di-segno: 12.52% enrolled
Le cose raccontano storie: 18.18% enrolled
Mini corso di ceramica: 30.59% enrolled
None: 20.03% enrolled


### RD38: _Count visitor education level breakdown from surveys_

In [60]:
pipeline = [
    {"$unwind": "$surveys"},
    {"$group": {
        "_id": "$surveys.title_of_studies",
        "count": {"$sum": 1}
    }},
    {"$sort": {"count": -1}}
]
result = list(db.visitors.aggregate(pipeline))
for study in result:
    print(f"{study['_id']}: {study['count']} visitors")

Laurea/PostLaurea: 24 visitors
Diploma: 19 visitors
None: 17 visitors
Lic. Media: 13 visitors
Elem.: 4 visitors
Nessuno: 3 visitors


### RD39: _Show workshop duration distribution_

In [61]:
pipeline = [
    {"$bucket": {
        "groupBy": "$duration",
        "boundaries": [0, 60, 120, 180, 240, 300],
        "default": "300+",
        "output": {
            "count": {"$sum": 1}
        }
    }}
]
result = list(db.activities.aggregate(pipeline))
for bucket in result:
    print(f"Duration {bucket['_id']}: {bucket['count']} activities")

Duration 60: 5 activities
Duration 120: 3 activities
Duration 240: 3 activities
Duration 300+: 14 activities


### RD40: Count the number of museum suppliers with external suppliers

In [65]:
pipeline = [
    {"$group": {
        "_id": "$isMuseum",
        "count": {"$sum": 1}
    }}
]
result = list(db.suppliers.aggregate(pipeline))
for item in result:
    label = "Museum" if item["_id"] else "Not a Museum"
    print(f"{label}: {item['count']} suppliers")

Museum: 15 suppliers
Not a Museum: 10 suppliers
